# TODO
* add a market index for qlib models
* write prediction process for kaggle under qlib framework

# imports

In [1]:
#  Copyright (c) Microsoft Corporation.
#  Licensed under the MIT License.
import sys, site
from pathlib import Path
# scripts_dir = Path.cwd().parent.joinpath("scripts")
scripts_dir = Path("/mnt/d/code/quant/qlib/scripts")
print(scripts_dir.joinpath("get_data.py"))
assert scripts_dir.joinpath("get_data.py").exists()
import numpy as np
import pandas as pd

/mnt/d/code/quant/qlib/scripts/get_data.py


In [2]:
sys.version

'3.8.13 (default, Mar 28 2022, 11:38:47) \n[GCC 7.5.0]'

In [3]:
################################# NOTE #################################
#  Please be aware that if colab installs the latest numpy and pyqlib  #
#  in this cell, users should RESTART the runtime in order to run the  #
#  following cells successfully.                                       #
########################################################################

try:
    import qlib
except ImportError:
    # install qlib
    ! pip install --upgrade numpy
    ! pip install pyqlib
    # reload
    site.main()

# scripts_dir = Path.cwd().parent.joinpath("scripts")
if not scripts_dir.joinpath("get_data.py").exists():
    # download get_data.py script
    scripts_dir = Path("~/tmp/qlib_code/scripts").expanduser().resolve()
    scripts_dir.mkdir(parents=True, exist_ok=True)
    import requests
    with requests.get("https://raw.githubusercontent.com/microsoft/qlib/main/scripts/get_data.py") as resp:
        with open(scripts_dir.joinpath("get_data.py"), "wb") as fp:
            fp.write(resp.content)

In [4]:
import qlib
import pandas as pd
from qlib.constant import REG_CN
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict


In [5]:
sys.path.append("/mnt/d/code/quant/qlib/scripts")
print(sys.path)

['/mnt/d/code/learn/pylearn/qlibs', '/home/jeffye/anaconda3/envs/py38/lib/python38.zip', '/home/jeffye/anaconda3/envs/py38/lib/python3.8', '/home/jeffye/anaconda3/envs/py38/lib/python3.8/lib-dynload', '', '/home/jeffye/anaconda3/envs/py38/lib/python3.8/site-packages', '/home/jeffye/anaconda3/envs/py38/lib/python3.8/site-packages/wheel-0.37.1-py3.8.egg', '/home/jeffye/anaconda3/envs/py38/lib/python3.8/site-packages/yelib-0.1.0-py3.8.egg', '/mnt/d/code/quant/qlib/scripts']


# explore kaggle jpx data
* "start_time": "2017-01-04", "end_time": "2022-02-28",

In [6]:
# use default data
# NOTE: need to download data from remote: python scripts/get_data.py qlib_data_cn --target_dir ~/.qlib/qlib_data/cn_data
provider_uri = "~/.qlib/qlib_data/kaggle"  # target_dir
# if not exists_qlib_data(provider_uri):
#     print(f"Qlib data is not found in {provider_uri}")
#     sys.path.append(str(scripts_dir))
#     from get_data import GetData
#     GetData().qlib_data(target_dir=provider_uri, region=REG_CN)
qlib.init(provider_uri=provider_uri, region=REG_CN)

[39648:MainThread](2022-06-05 14:43:58,353) INFO - qlib.Initialization - [config.py:402] - default_conf: client.
[39648:MainThread](2022-06-05 14:43:58,359) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[39648:MainThread](2022-06-05 14:43:58,362) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/home/jeffye/.qlib/qlib_data/kaggle')}


In [7]:
market = "all"
benchmark = "JPX1301"

In [8]:
from qlib.data import D
# D.calendar(start_time='2010-01-01', end_time='2017-12-31', freq='day')[:2]
from qlib.data.filter import ExpressionDFilter
expressionDFilter = ExpressionDFilter(rule_expression='$close>100')
instruments = D.instruments(market='all', filter_pipe=[expressionDFilter])
# D.list_instruments(instruments=instruments, start_time='2017-01-04', end_time='2022-02-28', as_list=True)

In [9]:
instruments = ['JPX1301', 'JPX1332']
fields = ['$close', '$volume', '$factor', 'Ref($close, 1)', 'Mean($close, 3)', '$high-$low']
f_d = D.features(instruments, fields, start_time='2017-01-04', end_time='2019-02-04', freq='day')
# f_d.isna().sum()
f_d

$close    $volume  $factor  Ref($close, 1)  \
instrument datetime                                                 
JPX1301    2017-01-04  2742.0    31400.0      1.0             NaN   
           2017-01-05  2738.0    17900.0      1.0          2742.0   
           2017-01-06  2740.0    19900.0      1.0          2738.0   
           2017-01-10  2748.0    24200.0      1.0          2740.0   
           2017-01-11  2745.0     9300.0      1.0          2748.0   
...                       ...        ...      ...             ...   
JPX1332    2019-01-29   675.0  1921600.0      1.0           663.0   
           2019-01-30   672.0  1520200.0      1.0           675.0   
           2019-01-31   670.0  1595700.0      1.0           672.0   
           2019-02-01   665.0  1420600.0      1.0           670.0   
           2019-02-04   673.0  1617900.0      1.0           665.0   

                       Mean($close, 3)  $high-$low  
instrument datetime                                 
JPX1301    2017-01-04      2742.000000        25.0  
           2017-01-05      2740.000000        12.0  
           2017-01-06      2740.000000        24.0  
           2017-01-10      2742.000000        19.0  
           2017-01-11      2744.333252        15.0  
...                                ...         ...  
JPX1332    2019-01-29       670.000000        12.0  
           2019-01-30       670.000000        11.0  
           2019-01-31       672.333313        15.0  
           2019-02-01       669.000000        12.0  
           2019-02-04       669.333313        16.0  

[1026 rows x 6 columns]

In [10]:
cols = fields
df =f_d
df[cols].groupby("datetime").rank(pct=True)

$close  $volume  $factor  Ref($close, 1)  \
instrument datetime                                               
JPX1301    2017-01-04     1.0      0.5     0.75             NaN   
           2017-01-05     1.0      0.5     0.75             1.0   
           2017-01-06     1.0      0.5     0.75             1.0   
           2017-01-10     1.0      0.5     0.75             1.0   
           2017-01-11     1.0      0.5     0.75             1.0   
...                       ...      ...      ...             ...   
JPX1332    2019-01-29     0.5      1.0     0.75             0.5   
           2019-01-30     0.5      1.0     0.75             0.5   
           2019-01-31     0.5      1.0     0.75             0.5   
           2019-02-01     0.5      1.0     0.75             0.5   
           2019-02-04     0.5      1.0     0.75             0.5   

                       Mean($close, 3)  $high-$low  
instrument datetime                                 
JPX1301    2017-01-04              1.0         1.0  
           2017-01-05              1.0         1.0  
           2017-01-06              1.0         1.0  
           2017-01-10              1.0         1.0  
           2017-01-11              1.0         1.0  
...                                ...         ...  
JPX1332    2019-01-29              0.5         0.5  
           2019-01-30              0.5         0.5  
           2019-01-31              0.5         0.5  
           2019-02-01              0.5         0.5  
           2019-02-04              0.5         0.5  

[1026 rows x 6 columns]

## remove 60 days for Alpha360 handler

In [11]:
df = f_d
df.index = df.index.get_level_values('datetime')
print(df.index.min(), df.index.max())


start_time = pd.to_datetime(df.index.min()) + pd.Timedelta(days=60)
print(start_time.strftime('%Y-%m-%d'))

2017-01-04 00:00:00 2019-02-04 00:00:00
2017-03-05


# train model

## define static parameters

In [12]:
experiment_name="online_srv"

In [13]:
###################################
# train model
###################################
# '2017-01-04', end_time='2022-02-28'
data_handler_config = {
    # "start_time": "2017-01-04",
    "start_time": start_time, # 
    "end_time": "2019-02-04",
    "fit_start_time": "2017-01-04",
    "fit_end_time": "2018-07-31",
    "instruments": market,
    "infer_processors": [
      {
        "class": "RobustZScoreNorm",
        "kwargs": {
          "fields_group": "feature",
          "clip_outlier": True
        }
      },
      {
        "class": "Fillna",
        "kwargs": {
          "fields_group": "feature"
        }
      }
    ],
    "learn_processors": [
      {
        "class": "DropnaLabel"
      },
      {
        "class": "CSRankNorm",
        "kwargs": {
          "fields_group": "label"
        }
      }
    ],
    "label": [
      "Ref($close, -2) / Ref($close, -1) - 1"
    ]
}

task = {   
    "model": {
        "class": "LSTM",
        "module_path": "qlib.contrib.model.pytorch_lstm",
        "kwargs": {
            "d_feat": 6,
            "hidden_size": 64,
            "num_layers": 2,
            "dropout": 0.1,
            "dec_dropout": 0.0,
            "n_epochs": 15,
            "lr": 1e-4,
            "early_stop": 3,
            "batch_size": 800,
            "metric": "loss",
            "loss": "mse",
            "optimizer": "adam",
            "GPU": 1
        },
    },
    "dataset": {
        "class": "DatasetH",
        "module_path": "qlib.data.dataset",
        "kwargs": {
            "handler": {
                "class": "Alpha360",
                "module_path": "qlib.contrib.data.handler",
                "kwargs": data_handler_config,
            },
            "segments": {
                "train": (start_time, "2018-07-31"),
                "valid": ("2018-08-01", "2019-02-04"),
                "test": ("2018-08-01", "2019-02-04"),
            },
        },
    },
}

# model initiaiton
model = init_instance_by_config(task["model"])
dataset = init_instance_by_config(task["dataset"])


Please install necessary libs for CatBoostModel.


/home/jeffye/anaconda3/envs/py38/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[39648:MainThread](2022-06-05 14:43:59,977) INFO - qlib.LSTM - [pytorch_lstm.py:58] - LSTM pytorch version...
[39648:MainThread](2022-06-05 14:44:00,298) INFO - qlib.LSTM - [pytorch_lstm.py:75] - LSTM parameters setting:
d_feat : 6
hidden_size : 64
num_layers : 2
dropout : 0.1
n_epochs : 15
lr : 0.0001
metric : loss
batch_size : 800
early_stop : 3
optimizer : adam
loss_type : mse
visible_GPU : 1
use_GPU : True
seed : None
[39648:MainThread](2022-06-05 14:44:59,384) INFO - qlib.timer - [log.py:113] - Time cost: 52.716s | Loading data Done
/home/jeffye/anaconda3/envs/py38/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1096: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanmedian

## explore Ahpha360 (DataHandlerLP) for data processing

In [14]:
dataset.handler._infer.iloc[-100].to_string()

'feature  CLOSE59                                  0.890904\n         CLOSE58                                  0.861238\n         CLOSE57                                  0.968804\n         CLOSE56                                  1.053052\n         CLOSE55                                  1.464344\n         CLOSE54                                  1.233618\n         CLOSE53                                  1.384717\n         CLOSE52                                  1.166845\n         CLOSE51                                  0.881984\n         CLOSE50                                  0.777057\n         CLOSE49                                  0.571961\n         CLOSE48                                  0.692527\n         CLOSE47                                  0.696914\n         CLOSE46                                  0.473196\n         CLOSE45                                  0.759398\n         CLOSE44                                  0.661038\n         CLOSE43                       

In [15]:
# dataset.handler._learn.iloc[-20:]
pd.set_option('display.max_rows', 400)
na_sum = dataset.handler._learn.isna().sum()

In [16]:
na_sum.to_string()

'feature  CLOSE59                                  0\n         CLOSE58                                  0\n         CLOSE57                                  0\n         CLOSE56                                  0\n         CLOSE55                                  0\n         CLOSE54                                  0\n         CLOSE53                                  0\n         CLOSE52                                  0\n         CLOSE51                                  0\n         CLOSE50                                  0\n         CLOSE49                                  0\n         CLOSE48                                  0\n         CLOSE47                                  0\n         CLOSE46                                  0\n         CLOSE45                                  0\n         CLOSE44                                  0\n         CLOSE43                                  0\n         CLOSE42                                  0\n         CLOSE41                             

In [17]:
dataset.handler._learn.loc[:, pd.IndexSlice[:, ['CLOSE0', 'OPEN0', 'LOW0', 'VOLUME0', 'HIGH0']]]

feature                                      
                       CLOSE0     OPEN0      LOW0 VOLUME0     HIGH0
datetime   instrument                                              
2017-03-06 JPX1301        0.0 -1.647025 -1.495574     0.0 -0.389995
           JPX1332        0.0  2.003291  0.574387     0.0  2.558551
           JPX1333        0.0 -0.488532 -0.082080     0.0 -0.839524
           JPX1376        0.0  0.348450  0.691736     0.0 -0.488260
           JPX1377        0.0  0.663846  0.640347     0.0  0.001264
...                       ...       ...       ...     ...       ...
2019-02-04 JPX9990        0.0 -0.922927 -0.376043     0.0  1.238996
           JPX9991        0.0 -2.371414 -3.000000     0.0 -0.521415
           JPX9993        0.0 -0.147190  0.823325     0.0  0.722399
           JPX9994        0.0 -2.001480 -2.281639     0.0 -0.610913
           JPX9997        0.0 -0.964642 -0.887971     0.0  1.965344

[890224 rows x 5 columns]

In [18]:
# dataset.handler._data.iloc[-20:].isna().sum().to_string()

# learn_ processors
csrank_norm = dataset.handler.learn_processors[1]

from qlib.data.dataset.processor import get_group_columns


print(csrank_norm.fields_group)
# cols = get_group_columns(df, csrank_norm.fields_group)
# print(cols)

label


In [19]:
# infer processor
robust_zscorenorm = dataset.handler.infer_processors[0]

_data = dataset.handler._data.copy()
print(robust_zscorenorm.fields_group)

feature


In [20]:
processor = dataset.handler.infer_processors
processor
# _infer_df = _shared_df.copy()
# _infer_df.iloc[:10]
dataset.handler._data.loc["2019-02-04"].iloc[-10:]

feature                                                    \
             CLOSE59   CLOSE58   CLOSE57   CLOSE56   CLOSE55   CLOSE54   
instrument                                                               
JPX9982     1.107771  1.095292  1.104935  1.096426  1.108905  1.108338   
JPX9983     1.197665  1.140681  1.156312  1.196082  1.208350  1.192917   
JPX9984     1.029224  1.034903  1.014553  1.012423  1.041765  1.039399   
JPX9987     0.987931  0.987931  1.025862  1.046552  1.056897  1.044828   
JPX9989     1.159888  1.152875  1.171108  1.150070  1.178121  1.180926   
JPX9990     0.996396  0.990090  0.990090  0.983784  0.990991  1.015315   
JPX9991     1.038314  1.040230  1.053640  1.047893  1.065134  1.070881   
JPX9993     0.928161  0.928161  0.927586  0.928736  0.930460  0.945402   
JPX9994     1.208296  1.204238  1.199279  1.193418  1.214608  1.206943   
JPX9997     1.222599  1.291525  1.265537  1.282486  1.325424  1.327684   

                                                    ...                      \
             CLOSE53   CLOSE52   CLOSE51   CLOSE50  ...   VOLUME8   VOLUME7   
instrument                                          ...                       
JPX9982     1.098695  1.094158  1.089053  1.091889  ...  0.254144  0.475138   
JPX9983     1.203007  1.189553  1.167392  1.173526  ...  2.137892  1.962599   
JPX9984     1.038452  1.057856  1.107667  1.077733  ...  1.541432  0.852809   
JPX9987     1.032759  1.013793  1.027586  1.024138  ...  1.295606  1.133600   
JPX9989     1.179523  1.158485  1.141655  1.140252  ...  2.359252  1.925256   
JPX9990     0.995495  0.987387  0.988288  0.985586  ...  0.597892  1.359940   
JPX9991     1.067050  1.045019  1.034483  1.039272  ...  0.230769  0.337469   
JPX9993     0.971839  0.967816  0.969540  0.979885  ...  0.607843  1.627451   
JPX9994     1.102795  1.154193  1.110009  1.128494  ...  0.492647  0.327206   
JPX9997     1.348023  1.323164  1.346893  1.332203  ...  0.483684  0.410684   

                                                                        \
             VOLUME6   VOLUME5   VOLUME4   VOLUME3   VOLUME2   VOLUME1   
instrument                                                               
JPX9982     0.408840  0.408840  0.751381  0.530387  0.552486  1.331492   
JPX9983     1.975633  1.116547  1.459955  1.129581  1.408387  1.836985   
JPX9984     0.978865  0.767552  1.126911  1.010345  1.841951  1.127458   
JPX9987     1.018198  0.678207  0.813582  1.088771  1.460719  1.122059   
JPX9989     1.598553  1.194093  2.958409  1.881857  2.025316  1.086799   
JPX9990     1.040663  0.384036  0.667169  1.067771  1.302711  0.745482   
JPX9991     0.265509  0.377171  0.265509  0.774194  1.429280  1.062035   
JPX9993     1.686275  0.352941  0.392157  1.019608  0.568627  0.431373   
JPX9994     1.000000  0.386029  0.672794  0.768382  0.397059  0.621324   
JPX9997     0.335992  0.493594  0.488760  0.647571  0.693498  1.940295   

                                                   label  
           VOLUME0 Ref($close, -2) / Ref($close, -1) - 1  
instrument                                                
JPX9982        1.0                             -0.001675  
JPX9983        1.0                              0.005092  
JPX9984        1.0                              0.005466  
JPX9987        1.0                              0.000000  
JPX9989        1.0                             -0.008415  
JPX9990        1.0                             -0.012635  
JPX9991        1.0                             -0.014327  
JPX9993        1.0                              0.007995  
JPX9994        1.0                              0.006685  
JPX9997        1.0                              0.030892  

[10 rows x 361 columns]

In [21]:
cols = ['CLOSE0', 'OPEN0', 'HIGH0', 'LOW0', 'VWAP0', 'VOLUME0']
t_temp = dataset.handler._data.loc[:, (['feature'], cols)]

t_temp
# print(t_temp.isna().sum())
# print(t_temp.sum())

feature                                            
                       CLOSE0     OPEN0     HIGH0      LOW0 VWAP0 VOLUME0
datetime   instrument                                                    
2017-03-06 JPX1301        1.0  0.980707  1.004823  0.980707   NaN     1.0
           JPX1332        1.0  1.023466  1.027076  0.996390   NaN     1.0
           JPX1333        1.0  0.994278  1.001431  0.991416   NaN     1.0
           JPX1376        1.0  1.004082  1.004082  0.997279   NaN     1.0
           JPX1377        1.0  1.007776  1.007776  0.996890   NaN     1.0
...                       ...       ...       ...       ...   ...     ...
2019-02-04 JPX9990        1.0  0.989189  1.017117  0.989189   NaN     1.0
           JPX9991        1.0  0.972222  1.003831  0.968391   NaN     1.0
           JPX9993        1.0  0.998276  1.013218  0.998276   NaN     1.0
           JPX9994        1.0  0.976555  1.003156  0.974752   NaN     1.0
           JPX9997        1.0  0.988701  1.022599  0.985311   NaN     1.0

[893587 rows x 6 columns]

In [22]:
t_temp = dataset.handler._data.loc["2019-02-04"]
cols = ['CLOSE4', 'CLOSE3', 'CLOSE2', 'CLOSE1', 'CLOSE0']
# cols = ['VOLUME4', 'VOLUME3','VOLUME2','VOLUME1','VOLUME0']

t1 = t_temp.loc[:, (['feature'], cols)]
print('shape', t1.shape)

nan_rows = t1[pd.isnull(t1[('feature', cols[1])])]

print(nan_rows.isna().sum())
print(nan_rows)

insts = ['JPX6824', 'JPX9535', 'JPX2790', 'JPX9982', 'JPX2226']

t1.isna().sum()
t1.loc[insts]


shape (1930, 5)
feature  CLOSE4     4
         CLOSE3    11
         CLOSE2     8
         CLOSE1     5
         CLOSE0     4
dtype: int64
             feature                                  
              CLOSE4 CLOSE3    CLOSE2    CLOSE1 CLOSE0
instrument                                            
JPX1981          NaN    NaN       NaN       NaN    NaN
JPX2226     1.011179    NaN  1.008130  1.013211    1.0
JPX2761     0.958730    NaN       NaN  0.976190    1.0
JPX2923     0.976303    NaN  0.969984  0.982622    1.0
JPX4365          NaN    NaN       NaN       NaN    NaN
JPX4628     1.023810    NaN       NaN  1.015152    1.0
JPX4781          NaN    NaN       NaN       NaN    NaN
JPX4966     1.031496    NaN       NaN  1.006299    1.0
JPX5945          NaN    NaN       NaN       NaN    NaN
JPX9539     1.000342    NaN       NaN       NaN    1.0
JPX9640     0.960279    NaN  0.977003  0.977700    1.0


feature                                     
              CLOSE4    CLOSE3    CLOSE2    CLOSE1 CLOSE0
instrument                                               
JPX6824     0.988201  0.997050  0.985251       NaN    1.0
JPX9535     1.014793  1.005917  1.002959  0.991124    1.0
JPX2790     1.001178  0.978210       NaN       NaN    1.0
JPX9982     1.007941  1.007941  0.996597  0.972774    1.0
JPX2226     1.011179       NaN  1.008130  1.013211    1.0

In [23]:
t_temp = dataset.handler._infer.loc["2019-02-04"]
t1 = t_temp.loc[:, (['feature'], ['CLOSE4', 'CLOSE3', 'CLOSE2', 'CLOSE1', 'CLOSE0'])]

# insts = ['6824', '9535', '2790', '9982']

t1.isna().sum()
t1.loc[insts]


feature                                     
              CLOSE4    CLOSE3    CLOSE2    CLOSE1 CLOSE0
instrument                                               
JPX6824    -0.353115 -0.069895 -0.726091  0.000000    0.0
JPX9535     0.585218  0.298132  0.187378 -0.673491    0.0
JPX2790     0.104800 -0.851872  0.000000  0.000000    0.0
JPX9982     0.343442  0.382131 -0.140802 -2.065937    0.0
JPX2226     0.457693  0.000000  0.454155  1.002480    0.0

In [24]:
t_temp = dataset.handler._infer.loc["2019-02-04"]
cols = ['CLOSE0', 'OPEN0', 'HIGH0', 'LOW0', 'VWAP0', 'VOLUME0']
t1 = t_temp.loc[:, (['feature'], cols)]

t1.loc[insts]

feature                                            
            CLOSE0     OPEN0     HIGH0      LOW0 VWAP0 VOLUME0
instrument                                                    
JPX6824        0.0 -0.251830 -0.638224  0.661541   0.0     0.0
JPX9535        0.0  0.505156 -0.245038  0.269881   0.0     0.0
JPX2790        0.0  0.000000 -1.029088  1.050896   0.0     0.0
JPX9982        0.0 -2.324335  2.954295 -2.542764   0.0     0.0
JPX2226        0.0  1.561660  1.394757  0.514343   0.0     0.0

In [25]:
print(dataset.handler._data.isna().sum().sum())
print(dataset.handler._data.shape)

56447823
(893587, 361)


In [26]:
dataset.handler._infer

feature                                          \
                        CLOSE59   CLOSE58   CLOSE57   CLOSE56   CLOSE55   
datetime   instrument                                                     
2017-03-06 JPX1301     0.000000  0.000000  0.000000  0.000000  0.000000   
           JPX1332     0.000000  0.000000  0.000000  0.000000  0.000000   
           JPX1333     0.000000  0.000000  0.000000  0.000000  0.000000   
           JPX1376     0.000000  0.000000  0.000000  0.000000  0.000000   
           JPX1377     0.000000  0.000000  0.000000  0.000000  0.000000   
...                         ...       ...       ...       ...       ...   
2019-02-04 JPX9990     0.185727  0.131810  0.129290  0.072660  0.131338   
           JPX9991     0.529986  0.547992  0.662593  0.616155  0.766320   
           JPX9993    -0.374673 -0.382229 -0.395236 -0.394020 -0.387068   
           JPX9994     1.925999  1.909329  1.884777  1.849867  2.046456   
           JPX9997     2.043467  2.633852  2.440808  2.604959  2.995516   

                                                                         ...  \
                        CLOSE54   CLOSE53   CLOSE52   CLOSE51   CLOSE50  ...   
datetime   instrument                                                    ...   
2017-03-06 JPX1301     0.000000  0.000000  0.000000  0.000000  0.000000  ...   
           JPX1332     0.000000  0.000000  0.000000  0.000000  0.000000  ...   
           JPX1333     0.000000  0.000000  0.000000  0.000000  0.000000  ...   
           JPX1376     0.000000  0.000000  0.000000  0.000000  0.000000  ...   
           JPX1377     0.000000  0.000000  0.000000  0.000000  0.000000  ...   
...                         ...       ...       ...       ...       ...  ...   
2019-02-04 JPX9990     0.339102  0.165893  0.090879  0.096299  0.068879  ...   
           JPX9991     0.819685  0.790186  0.598928  0.507567  0.551410  ...   
           JPX9993    -0.265566 -0.040503 -0.081650 -0.070615  0.017643  ...   
           JPX9994     1.996467  1.102056  1.561343  1.179976  1.353332  ...   
           JPX9997     3.000000  3.000000  3.000000  3.000000  3.000000  ...   

                                                                         \
                        VOLUME8   VOLUME7   VOLUME6   VOLUME5   VOLUME4   
datetime   instrument                                                     
2017-03-06 JPX1301    -0.876204 -1.222551 -1.215429 -0.976274 -1.082677   
           JPX1332     1.151866 -0.140376 -0.027432 -0.094221 -0.358237   
           JPX1333    -0.149580  0.153647 -0.142694 -0.192526  0.012172   
           JPX1376     0.906746  0.018820  0.274167  1.174186  1.139464   
           JPX1377     0.176066 -0.266921  0.640752  1.143556  0.764250   
...                         ...       ...       ...       ...       ...   
2019-02-04 JPX9990    -0.665763  0.545548  0.034145 -1.070208 -0.604014   
           JPX9991    -1.245467 -1.092969 -1.232929 -1.081751 -1.290346   
           JPX9993    -0.650049  0.974237  1.089468 -1.122496 -1.073937   
           JPX9994    -0.831949 -1.109416 -0.032323 -1.066856 -0.594402   
           JPX9997    -0.846103 -0.975641 -1.117716 -0.885979 -0.908868   

                                                             \
                        VOLUME3   VOLUME2   VOLUME1 VOLUME0   
datetime   instrument                                         
2017-03-06 JPX1301    -0.638336 -1.064461 -1.348311     0.0   
           JPX1332     0.208999 -0.245410 -0.070249     0.0   
           JPX1333    -0.311592  0.026917  0.370454     0.0   
           JPX1376     0.185355 -0.094121  1.352881     0.0   
           JPX1377     0.743514  0.989553  0.790632     0.0   
...                         ...       ...       ...     ...   
2019-02-04 JPX9990     0.084752  0.527653 -0.582860     0.0   
           JPX9991    -0.431330  0.763302  0.089939     0.0   
           JPX9993     0.000086 -0.839075 -1.250466     0.0   
           JPX9994    -0.441546 -1.158505 -0.

In [27]:
# start exp to train model

experiment_id = 'jpx'
# experiment_name: Optional[Text] = None,
# recorder_id: Optional[Text] = None,

# with R.start(experiment_name=experiment_name, experiment_id=experiment_id):
with R.start(experiment_name=experiment_name):
    R.log_params(**flatten_dict(task))
    model.fit(dataset)
    R.save_objects(trained_model=model)
    rid = R.get_recorder().id
    # prediction
    recorder = R.get_recorder()
    sr = SignalRecord(model, dataset, recorder)
    sr.generate()


[39648:MainThread](2022-06-05 14:47:03,458) INFO - qlib.workflow - [expm.py:315] - <mlflow.tracking.client.MlflowClient object at 0x7fd553fb2760>
[39648:MainThread](2022-06-05 14:47:03,506) INFO - qlib.workflow - [exp.py:257] - Experiment 0 starts running ...
[39648:MainThread](2022-06-05 14:47:04,158) INFO - qlib.workflow - [recorder.py:293] - Recorder 0b1cb59dbdd14236abc2f7c2ea6cfda4 starts running under Experiment 0 ...
[39648:MainThread](2022-06-05 14:47:07,264) INFO - qlib.LSTM - [pytorch_lstm.py:236] - training...
[39648:MainThread](2022-06-05 14:47:07,266) INFO - qlib.LSTM - [pytorch_lstm.py:240] - Epoch0:
[39648:MainThread](2022-06-05 14:47:07,267) INFO - qlib.LSTM - [pytorch_lstm.py:241] - training...
[39648:MainThread](2022-06-05 14:47:35,632) INFO - qlib.LSTM - [pytorch_lstm.py:243] - evaluating...
[39648:MainThread](2022-06-05 14:47:47,861) INFO - qlib.LSTM - [pytorch_lstm.py:246] - train -0.997211, valid -0.997335
[39648:MainThread](2022-06-05 14:47:47,863) INFO - qlib.LST

'The following are prediction results of the LSTM model.'
                          score
datetime   instrument          
2018-08-01 JPX1301     0.012654
           JPX1332     0.020338
           JPX1333     0.010547
           JPX1376     0.018653
           JPX1377     0.019007


In [28]:
print(rid, experiment_name)

0b1cb59dbdd14236abc2f7c2ea6cfda4 online_srv


# prediction, backtest & analysis

In [29]:
###################################
# prediction, backtest & analysis
###################################
port_analysis_config = {
    "executor": {
        "class": "SimulatorExecutor",
        "module_path": "qlib.backtest.executor",
        "kwargs": {
            "time_per_step": "day",
            "generate_portfolio_metrics": True,
        },
    },
    "strategy": {
        "class": "TopkDropoutStrategy",
        "module_path": "qlib.contrib.strategy.signal_strategy",
        "kwargs": {
            "model": model,
            "dataset": dataset,
            "topk": 200,
            "n_drop": 20,
        },
    },
    "backtest": {
        "start_time": "2021-12-06",
        "end_time": "2022-02-27",
        "account": 100000000,
        "benchmark": benchmark,
        "exchange_kwargs": {
            "freq": "day",
            "limit_threshold": 0.095,
            "deal_price": "close",
            "open_cost": 0.0005,
            "close_cost": 0.0015,
            "min_cost": 5,
        },
    },
}

# backtest and analysis
backtest_experiment_name = "backtest1"
with R.start(experiment_name=backtest_experiment_name, experiment_id='back0', resume=True): 
    recorder = R.get_recorder(recorder_id=rid, experiment_name=experiment_name)
    model = recorder.load_object("trained_model")

    # prediction
    recorder = R.get_recorder()
    ba_rid = recorder.id
    sr = SignalRecord(model, dataset, recorder)
    sr.generate()

    # # backtest & analysis
    # par = PortAnaRecord(recorder, port_analysis_config, "day")
    # par.generate()


[39648:MainThread](2022-06-05 14:50:43,879) INFO - qlib.workflow - [expm.py:315] - <mlflow.tracking.client.MlflowClient object at 0x7fd52d8ffc70>
[39648:MainThread](2022-06-05 14:50:43,891) WARNING - qlib.workflow - [expm.py:195] - No valid experiment found. Create a new experiment with name backtest1.
[39648:MainThread](2022-06-05 14:50:43,944) INFO - qlib.workflow - [exp.py:257] - Experiment 1 starts running ...
[39648:MainThread](2022-06-05 14:50:43,945) WARNING - qlib.workflow - [exp.py:307] - Please make sure the recorder name mlflow_recorder is unique, we will only return the latest recorder if there exist several matched the given name.
[39648:MainThread](2022-06-05 14:50:43,960) INFO - qlib.workflow - [exp.py:193] - No valid recorder found. Create a new recorder with name mlflow_recorder.
[39648:MainThread](2022-06-05 14:50:44,330) INFO - qlib.workflow - [recorder.py:293] - Recorder 8f5d57252a004c89af06b91df89dc199 starts running under Experiment 1 ...
/home/jeffye/anaconda3/en

'The following are prediction results of the LSTM model.'
                          score
datetime   instrument          
2018-08-01 JPX1301     0.012654
           JPX1332     0.020338
           JPX1333     0.010547
           JPX1376     0.018653
           JPX1377     0.019007


In [30]:
recorder = R.get_recorder(experiment_name=experiment_name)
recorder.load_object('pred.pkl').index.get_level_values('datetime')

[39648:MainThread](2022-06-05 14:50:48,525) WARNING - qlib.workflow - [exp.py:307] - Please make sure the recorder name mlflow_recorder is unique, we will only return the latest recorder if there exist several matched the given name.


DatetimeIndex(['2018-08-01', '2018-08-01', '2018-08-01', '2018-08-01',
               '2018-08-01', '2018-08-01', '2018-08-01', '2018-08-01',
               '2018-08-01', '2018-08-01',
               ...
               '2019-02-04', '2019-02-04', '2019-02-04', '2019-02-04',
               '2019-02-04', '2019-02-04', '2019-02-04', '2019-02-04',
               '2019-02-04', '2019-02-04'],
              dtype='datetime64[ns]', name='datetime', length=238144, freq=None)

In [31]:
R.get_recorder(experiment_name=experiment_name)

[39648:MainThread](2022-06-05 14:50:49,631) WARNING - qlib.workflow - [exp.py:307] - Please make sure the recorder name mlflow_recorder is unique, we will only return the latest recorder if there exist several matched the given name.


MLflowRecorder(info={'class': 'Recorder', 'id': '0b1cb59dbdd14236abc2f7c2ea6cfda4', 'name': 'mlflow_recorder', 'experiment_id': '0', 'start_time': '2022-06-05 14:47:03', 'end_time': '2022-06-05 14:50:43', 'status': 'FINISHED'},
               uri=file:/mnt/d/code/learn/pylearn/qlibs/mlruns,
               artifact_uri=None,
               client=<mlflow.tracking.client.MlflowClient object at 0x7fd51ebf8130>)

In [32]:
# 模型的predict score，ADD模型： ADDModel.forward() 输出"excess"，["market"]，["adv_market"]，["adv_excess"]，"reconstructed_feature"
# ADD.predict() 取的是“excess”, 成为pred.pkl 中的score.

# SignalRecord中实现如下：
def generate(self, **kwargs):
    # generate prediciton
    pred = self.model.predict(self.dataset)
    if isinstance(pred, pd.Series):
        pred = pred.to_frame("score")
    self.save(**{"pred.pkl": pred})

    logger.info(
        f"Signal record 'pred.pkl' has been saved as the artifact of the Experiment {self.recorder.experiment_id}"
    )
    # print out results
    pprint(f"The following are prediction results of the {type(self.model).__name__} model.")
    pprint(pred.head(5))

    if isinstance(self.dataset, DatasetH):
        raw_label = self.generate_label(self.dataset)
        self.save(**{"label.pkl": raw_label})

pred_score = par.load("pred.pkl")
pred_score.head(2)

NameError: name 'par' is not defined

In [ ]:
s = pd.Series(["a", "b", "c"], name='vals')
print(s)
s.to_frame()

# analyze graphs

In [ ]:
from pprint import pprint

import qlib
import pandas as pd
from qlib.utils.time import Freq
from qlib.utils import flatten_dict
from qlib.contrib.evaluate import backtest_daily
from qlib.contrib.evaluate import risk_analysis
from qlib.contrib.strategy import TopkDropoutStrategy

CSI300_BENCH = benchmark
STRATEGY_CONFIG = {
    "topk": 50,
    "n_drop": 5,
    # pred_score, pd.Series
    "signal": pred_score,
}


strategy_obj = TopkDropoutStrategy(**STRATEGY_CONFIG)
report_normal, positions_normal = backtest_daily(
    start_time="2021-12-06", end_time="2022-02-27", strategy=strategy_obj
)
analysis = dict()
# default frequency will be daily (i.e. "day")
analysis["excess_return_without_cost"] = risk_analysis(report_normal["return"] - report_normal["bench"])
analysis["excess_return_with_cost"] = risk_analysis(report_normal["return"] - report_normal["bench"] - report_normal["cost"])

analysis_df = pd.concat(analysis)  # type: pd.DataFrame
pprint(analysis_df)

In [ ]:
# report_normal, positions_normal = backtest_daily(
#     start_time="2017-01-01", end_time="2020-08-01", strategy=strategy_obj
# )
from typing import Union
from qlib.backtest import backtest
from qlib.backtest.executor import SimulatorExecutor

start_time = "2017-01-01"
end_time = "2020-08-01"
strategy = strategy_obj
executor = None
account = 1.e8
benchmark: str = benchmark
exchange_kwargs: dict = None
pos_type: str = "Position"

freq = "day"
if executor is None:
    executor_config = {
        "time_per_step": freq,
        "generate_portfolio_metrics": True,
    }
    executor = SimulatorExecutor(**executor_config)
    
_exchange_kwargs = {
    "freq": freq,
    "limit_threshold": None,
    "deal_price": None,
    "open_cost": 0.0005,
    "close_cost": 0.0015,
    "min_cost": 5,
}
# if exchange_kwargs is not None:
#     _exchange_kwargs.update(exchange_kwargs)

print(account)

portfolio_metric_dict, indicator_dict = backtest(
    start_time=start_time,
    end_time=end_time,
    strategy=strategy,
    executor=executor,
    account=account,
    benchmark=benchmark,
    exchange_kwargs=_exchange_kwargs,
    pos_type=pos_type,
)



# analysis_freq = "{0}{1}".format(*Freq.parse(freq))

# report_normal, positions_normal = portfolio_metric_dict.get(analysis_freq)

In [ ]:
strategy.signal.signal_cache.loc['2017-01-10']
# pred_score.loc['2017-01-04']
print('position type', type(strategy.common_infra.get("trade_account").current_position))
position_dict = strategy.common_infra.get("trade_account").current_position.position


print(len(position_dict), position_dict)  # 默认维护52只股票 + cash + now_account_value
# 单只股票信息：
# 'SZ000001': {'amount': 392925.71123170934, 'price': 8.12631893157959, 'weight': 0.016886521190966564, 'count_day': 55}

# executor.track_data

In [ ]:
# portfolio_metric_dict --> 计算在account类中, def _update_state_from_order(self, order, trade_val, cost, trade_price) --》AccumulatedInfo()
# report 中也记录了turnover,
# turnover 定义：https://www.investopedia.com/terms/p/portfolioturnover.asp

# type(portfolio_metric_dict)
executor.trade_account.accum_info.to

In [ ]:
# portfolio_metric_dict 的具体计算方法见：account.update_portfolio_metrics, 计算依据position 类中记录的信息qlib.backtest.position.py
# SimulatorExecutor 中维护（生成）： 1. self.level_infra = LevelInfrastructure() -->def reset_common_infra L117 生成trade_account --> 
# account中维护position 

# portfolio_metric_dict

# executor 和 strategy中使用/维护的是同一个account 和 同一个common_infra
print(executor.trade_account)
strategy.common_infra.get("trade_account")
print(strategy.common_infra, executor.common_infra)


In [ ]:
# indicator_dict 主要也在account中计算。
# ffr: order fulfill rate,  pa: order price advantage, pos:positive_rate 更多参考 BaseOrderIndicator
# class Indicator:
#     """
#     `Indicator` is implemented in a aggregate way.
#     All the metrics are calculated aggregately.
#     All the metrics are calculated for a separated stock and in a specific step on a specific level.

#     | indicator    | desc.                                                        |
#     |--------------+--------------------------------------------------------------|
#     | amount       | the *target* amount given by the outer strategy              |
#     | deal_amount  | the real deal amount                                         |
#     | inner_amount | the total *target* amount of inner strategy                  |
#     | trade_price  | the average deal price                                       |
#     | trade_value  | the total trade value                                        |
#     | trade_cost   | the total trade cost  (base price need drection)             |
#     | trade_dir    | the trading direction                                        |
#     | ffr          | full fill rate                                               |
#     | pa           | price advantage                                              |
#     | pos          | win rate                                                     |
#     | base_price   | the price of baseline                                        |
#     | base_volume  | the volume of baseline (for weighted aggregating base_price) |

#     **NOTE**:
#     The `base_price` and `base_volume` can't be NaN when there are not trading on that step. Otherwise
#     aggregating get wrong results.

#     So `base_price` will not be calculated in a aggregate way!!

#     """
indicator_dict

In [ ]:
from qlib.contrib.report import analysis_model, analysis_position
from qlib.data import D
recorder = R.get_recorder(recorder_id=ba_rid, experiment_name="backtest_analysis")
print(recorder)
pred_df = recorder.load_object("pred.pkl")
report_normal_df = recorder.load_object("portfolio_analysis/report_normal_1day.pkl")
positions = recorder.load_object("portfolio_analysis/positions_normal_1day.pkl")
analysis_df = recorder.load_object("portfolio_analysis/port_analysis_1day.pkl")

## analysis position

### report

In [ ]:
analysis_position.report_graph(report_normal_df)

### risk analysis

In [ ]:
analysis_position.risk_analysis_graph(analysis_df, report_normal_df)

## analysis model

In [ ]:
label_df = dataset.prepare("test", col_set="label")
label_df.columns = ['label']

### score IC

In [ ]:
pred_label = pd.concat([label_df, pred_df], axis=1, sort=True).reindex(label_df.index)
analysis_position.score_ic_graph(pred_label)

### model performance

In [ ]:
analysis_model.model_performance_graph(pred_label)

# jpx prediction

In [33]:
import sys
sys.path.append("/mnt/d/dataset/quant/kaggle22/")

import jpx_tokyo_market_prediction
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

<frozen importlib._bootstrap>:219: RuntimeWarning: compiletime version 3.7 of module 'jpx_tokyo_market_prediction.competition' does not match runtime version 3.8


In [35]:
prices, options, financials, trades, secondary_prices, sample_prediction = next(iter_test)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


In [36]:
prices
start_date = prices['Date'].min()
end_date = prices['Date'].max()
print(type(start_date), start_date, end_date)

prices = prices.rename(columns={"SecuritiesCode": "instrument", "Open": "open", "High": "high", "Low": "low", "Close": "close", "Volume": "volume", "AdjustmentFactor": "factor", "Date": "date"})

<class 'str'> 2021-12-06 2021-12-06


In [37]:
prices['date'] = pd.to_datetime(prices['date'])
prices['instrument'] = prices['instrument'].apply(lambda x: 'jpx' + str(x))

In [38]:
# prices.info()
prices.dtypes

date                datetime64[ns]
RowId                       object
instrument                  object
open                       float64
high                       float64
low                        float64
close                      float64
volume                       int64
factor                     float64
ExpectedDividend           float64
SupervisionFlag               bool
dtype: object

In [45]:
from dump_bin import DumpDataBase, DumpDataUpdate
from typing import Union
from loguru import logger
from qlib.utils import fname_to_code, code_to_fname
import abc
import shutil
import traceback
from pathlib import Path
from typing import Iterable, List, Union
from functools import partial
from concurrent.futures import ThreadPoolExecutor, as_completed, ProcessPoolExecutor

import fire
import numpy as np
import pandas as pd
from tqdm import tqdm
from loguru import logger
from qlib.utils import fname_to_code, code_to_fname

class DumpDataUpdateInMemory(DumpDataUpdate):
    def __init__(
        self,
        data : pd.DataFrame,
        csv_path: str,
        qlib_dir: str,
        backup_dir: str = None,
        freq: str = "day",
        max_workers: int = 16,
        date_field_name: str = "date",
        file_suffix: str = ".csv",
        symbol_field_name: str = "symbol",
        exclude_fields: str = "",
        include_fields: str = "",
        limit_nums: int = None,
    ):
        self.data = data
        super().__init__(
            csv_path,
            qlib_dir,
            backup_dir,
            freq,
            max_workers,
            date_field_name,
            file_suffix,
            symbol_field_name,
            exclude_fields,
            include_fields,
        )

    def _load_all_source_data(self):
        return self.data
    

    def save_instruments(self, instruments_data: Union[list, pd.DataFrame]):
        self._instruments_dir.mkdir(parents=True, exist_ok=True)
        instruments_path = str(self._instruments_dir.joinpath(self.INSTRUMENTS_FILE_NAME).resolve())
        if isinstance(instruments_data, pd.DataFrame):
            _df_fields = [self.symbol_field_name, self.INSTRUMENTS_START_FIELD, self.INSTRUMENTS_END_FIELD]
            instruments_data = instruments_data.loc[:, _df_fields]
            logger.info(f'{self.symbol_field_name}, with {instruments_data.info()}')
            instruments_data[self.symbol_field_name] = instruments_data[self.symbol_field_name].astype(str).apply(
                lambda x: fname_to_code(x.lower()).upper()
            )
            instruments_data.to_csv(instruments_path, header=False, sep=self.INSTRUMENTS_SEP, index=False)
        else:
            np.savetxt(instruments_path, instruments_data, fmt="%s", encoding="utf-8")
    

    def _dump_features(self):
        logger.info("start dump features......")
        error_code = {}
        with ProcessPoolExecutor(max_workers=self.works) as executor:
            futures = {}
            i = 0 
            for _code, _df in self._all_data.groupby(self.symbol_field_name):
                _code = fname_to_code(str(_code).lower()).upper()
                _start, _end = self._get_date(_df, is_begin_end=True)
                if not (isinstance(_start, pd.Timestamp) and isinstance(_end, pd.Timestamp)):
                    continue
                if _code in self._update_instruments:
                    # exists stock, will append data
                    _update_calendars = (
                        _df[_df[self.date_field_name] > self._update_instruments[_code][self.INSTRUMENTS_END_FIELD]][
                            self.date_field_name
                        ]
                        .sort_values()
                        .to_list()
                    )
                    if i < 2:
                        logger.info(f"{_code}: {_update_calendars}")
                        i += 2
                    if _update_calendars:
                        self._update_instruments[_code][self.INSTRUMENTS_END_FIELD] = self._format_datetime(_end)
                        futures[executor.submit(self._dump_bin, _df, _update_calendars)] = _code
                else:
                    # new stock
                    _dt_range = self._update_instruments.setdefault(_code, dict())
                    _dt_range[self.INSTRUMENTS_START_FIELD] = self._format_datetime(_start)
                    _dt_range[self.INSTRUMENTS_END_FIELD] = self._format_datetime(_end)
                    futures[executor.submit(self._dump_bin, _df, self._new_calendar_list)] = _code
                

            with tqdm(total=len(futures)) as p_bar:
                for _future in as_completed(futures):
                    try:
                        _future.result()
                    except Exception:
                        error_code[futures[_future]] = traceback.format_exc()
                    p_bar.update()
            logger.info(f"dump bin errors： {error_code}")

        logger.info("end of features dump.\n")
    

    def _data_to_bin(self, df: pd.DataFrame, calendar_list: List[pd.Timestamp], features_dir: Path):
        if df.empty:
            logger.warning(f"{features_dir.name} data is None or empty")
            return
        if not calendar_list:
            logger.warning("calendar_list is empty")
            return
        # align index
        _df = self.data_merge_calendar(df, calendar_list)
        # used when creating a bin file
        date_index = self.get_datetime_index(_df, calendar_list)
        for field in self.get_dump_fields(_df.columns):
            bin_path = features_dir.joinpath(f"{field.lower()}.{self.freq}{self.DUMP_FILE_SUFFIX}")

            # logger.info(f"update field={field}, columns={_df.columns} to {bin_path}, mode={self._mode}, date_index={date_index}, data={_df[field]}")
            if field not in _df.columns:
                continue
            if bin_path.exists() and self._mode == self.UPDATE_MODE:
                # update
                with bin_path.open("ab") as fp:
                    np.array(_df[field]).astype("<f").tofile(fp)
            else:
                # append; self._mode == self.ALL_MODE or not bin_path.exists()
                np.hstack([date_index, _df[field]]).astype("<f").tofile(str(bin_path.resolve()))


    def _dump_bin(self, file_or_data: [Path, pd.DataFrame], calendar_list: List[pd.Timestamp]):
        if not calendar_list:
            logger.warning("calendar_list is empty")
            return
        if isinstance(file_or_data, pd.DataFrame):
            if file_or_data.empty:
                return
            code = fname_to_code(str(file_or_data.iloc[0][self.symbol_field_name]).lower())
            df = file_or_data
        elif isinstance(file_or_data, Path):
            code = self.get_symbol_from_file(file_or_data)
            df = self._get_source_data(file_or_data)
        else:
            raise ValueError(f"not support {type(file_or_data)}")
        if df is None or df.empty:
            logger.warning(f"{code} data is None or empty")
            return

        # try to remove dup rows or it will cause exception when reindex.
        df = df.drop_duplicates(self.date_field_name)

        # features save dir
        features_dir = self._features_dir.joinpath(code_to_fname(code).lower())
        features_dir.mkdir(parents=True, exist_ok=True)
        self._data_to_bin(df, calendar_list, features_dir)


In [ ]:
_dump = DumpDataUpdateInMemory(data=prices, csv_path='~/.', qlib_dir=provider_uri, symbol_field_name='instrument', exclude_fields="RowId,SupervisionFlag")

for _code, _df in prices.groupby(_dump.symbol_field_name):
    _code = _code.upper()
    _start, _end = _dump._get_date(_df, is_begin_end=True)
    print(_start, _end, type(_start), type(_end))
    if not (isinstance(_start, pd.Timestamp) and isinstance(_end, pd.Timestamp)):
        print('skipped')
    # print(_dump._update_instruments)
    if _code in _dump._update_instruments:
        # exists stock, will append data
        print('end date', type(_dump._update_instruments[_code][_dump.INSTRUMENTS_END_FIELD]))
        print('_df', _df[_dump.date_field_name])
        _update_calendars = (
            # _df[_df[_dump.date_field_name] > _dump._update_instruments[_code][_dump.INSTRUMENTS_END_FIELD]][
            #     _dump.date_field_name
            # ]
            _df[_df[_dump.date_field_name] > "2021-12-03"][
                _dump.date_field_name
            ]
            .sort_values()
            .to_list()
        )
        # print(_update_calendars)
        # print('origin', _df)
        # print('merged', _dump.data_merge_calendar(_df, _update_calendars))
        print(_dump.get_dump_fields(_df.columns))

        features_dir = _dump._features_dir.joinpath(_code.lower())
        features_dir.mkdir(parents=True, exist_ok=True)
        print(features_dir)

        _df = _dump.data_merge_calendar(_df, _update_calendars)
        # _dump._data_to_bin(_df, _update_calendars, features_dir)
        # print(_df.columns)
        # print(_dump.UPDATE_MODE, _dump._mode)
        date_index = _dump.get_datetime_index(_df, _update_calendars)
        print('date_index', date_index)
        for field in _dump.get_dump_fields(_df.columns):
            bin_path = features_dir.joinpath(f"{field.lower()}.{_dump.freq}{_dump.DUMP_FILE_SUFFIX}")
            if field not in _df.columns:
                continue
            if bin_path.exists() and _dump._mode == _dump.UPDATE_MODE:
                # update
                with bin_path.open("ab") as fp:
                    np.array(_df[field]).astype("<f").tofile(fp)
                print('begin append')
            else:
                # append; self._mode == self.ALL_MODE or not bin_path.exists()
                # np.hstack([date_index, _df[field]]).astype("<f").tofile(str(bin_path.resolve()))
                print('other')
        
    else:
        print('fuckedup')
        print(type(_code), type(list(_dump._update_instruments.keys())[0]))
    break

In [ ]:
_dump = DumpDataUpdateInMemory(data=prices, csv_path='~/.', qlib_dir=provider_uri, symbol_field_name='instrument', exclude_fields="instrument,RowId,SupervisionFlag,date,ExpectedDividend")
_dump.dump()

# print(_dump.symbol_field_name)

## problems
* 到底数据normalization 是怎么做的？ - 可以用中文数据分解每一步来验证，或者看看dump？
* def _get_calendar_list(self) 到底是什么作用？
* 

## qlib update_predictions

In [47]:
import copy
import fire
import qlib
from qlib.constant import REG_CN
from qlib.model.trainer import task_train
from qlib.workflow.online.utils import OnlineToolR
from qlib.tests.config import CSI300_GBDT_TASK

from qlib.workflow.online.update import DSBasedUpdater, RMDLoader, _replace_range

from typing import Optional

import pandas as pd
from qlib import get_module_logger
from qlib.data import D
from qlib.data.dataset import Dataset, DatasetH, TSDatasetH
from qlib.data.dataset.handler import DataHandlerLP
from qlib.model import Model
from qlib.utils import get_date_by_shift
from qlib.workflow.recorder import Recorder


class PredUpdater(DSBasedUpdater):
    def __init__(
            self,
            modle,
            record,
            to_date=None,
            from_date=None,
            hist_ref: Optional[int] = None,
            freq="day",
            fname="pred.pkl",
            loader_cls: type = RMDLoader,
        ):
        self.model = model
        super().__init__(record=record, to_date=to_date, from_date=from_date, hist_ref=hist_ref, freq=freq, fname=fname, loader_cls=loader_cls)

    def get_update_data(self, dataset) -> pd.DataFrame:
        # Load model
        model = self.model
        new_pred: pd.Series = model.predict(dataset)
        data = _replace_range(self.old_data, new_pred.to_frame("score"))
        self.logger.info(f"Finish updating new {new_pred.shape[0]} predictions in {self.record.info['id']}.")
        return data



In [ ]:
dataset

In [ ]:
# re-init the data provider to get updated data.
# also have to setup_data for handler to process new data.
# TODO: fix the efficiency problem since it takes more than 1m to just reload data for a single day. 
qlib.init(provider_uri=provider_uri, region=REG_CN)

dataset.handler.start_time = '2021-12-06'
dataset.handler.end_time = '2021-12-06'
dataset.segments['test'] = (dataset.handler.start_time, dataset.handler.end_time)
dataset.segments['train'] = dataset.segments['test']
dataset.segments['valid'] = dataset.segments['test']

# setup_data 继承过程中添加新功能， DataHandlerLP中重新process data
dataset.handler.setup_data(init_type="load_state") # only process the data, no fit.


[6202:MainThread](2022-06-03 22:59:32,689) INFO - qlib.Initialization - [config.py:402] - default_conf: client.
[6202:MainThread](2022-06-03 22:59:32,692) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[6202:MainThread](2022-06-03 22:59:32,693) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/home/jeffye/.qlib/qlib_data/kaggle')}
[6202:MainThread](2022-06-03 23:00:41,939) INFO - qlib.timer - [log.py:113] - Time cost: 69.244s | Loading data Done
[6202:MainThread](2022-06-03 23:00:42,518) INFO - qlib.timer - [log.py:113] - Time cost: 0.566s | RobustZScoreNorm Done
[6202:MainThread](2022-06-03 23:00:42,529) INFO - qlib.timer - [log.py:113] - Time cost: 0.009s | Fillna Done
[6202:MainThread](2022-06-03 23:00:42,537) INFO - qlib.timer - [log.py:113] - Time cost: 0.004s | DropnaLabel Done
/home/jeffye/anaconda3/envs/py38/lib/python3.8/site-packages/qlib/data/dataset/processor.py:352: SettingWithC

In [ ]:
print(dataset.handler._data.index.get_level_values('datetime').unique())
dataset.handler._infer

DatetimeIndex(['2021-12-01', '2021-12-02', '2021-12-03', '2021-12-06'], dtype='datetime64[ns]', name='datetime', freq=None)


feature                                          \
                        CLOSE59   CLOSE58   CLOSE57   CLOSE56   CLOSE55   
datetime   instrument                                                     
2021-12-01 JPX1301     0.593373  0.638925  0.714064  0.716875  0.661569   
           JPX1332     0.930717  0.951560  0.987314  0.948610  0.954512   
           JPX1333     0.990512  0.885610  1.398779  1.286901  1.460168   
           JPX1375     2.870617  2.877484  3.000000  3.000000  3.000000   
           JPX1376     1.004424  1.036134  1.204842  1.200138  1.233930   
...                         ...       ...       ...       ...       ...   
2021-12-06 JPX9990     0.000000  0.000000  0.000000  0.000000  0.000000   
           JPX9991     0.000000  0.000000  0.000000  0.000000  0.000000   
           JPX9993     0.000000  0.000000  0.000000  0.000000  0.000000   
           JPX9994     0.000000  0.000000  0.000000  0.000000  0.000000   
           JPX9997     0.000000  0.000000  0.000000  0.000000  0.000000   

                                                                         ...  \
                        CLOSE54   CLOSE53   CLOSE52   CLOSE51   CLOSE50  ...   
datetime   instrument                                                    ...   
2021-12-01 JPX1301     0.753324  0.846585  0.880408  0.763311  0.843738  ...   
           JPX1332     1.080570  1.208386  1.231044  1.162214  1.294524  ...   
           JPX1333     1.512061  1.745969  1.793368  1.614047  1.766143  ...   
           JPX1375     3.000000  3.000000  3.000000  3.000000  2.834213  ...   
           JPX1376     1.433976  1.540019  1.681077  1.694268  1.679816  ...   
...                         ...       ...       ...       ...       ...  ...   
2021-12-06 JPX9990     0.000000  0.000000  0.000000  0.000000  0.000000  ...   
           JPX9991     0.000000  0.000000  0.000000  0.000000  0.000000  ...   
           JPX9993     0.000000  0.000000  0.000000  0.000000  0.000000  ...   
           JPX9994     0.000000  0.000000  0.000000  0.000000  0.000000  ...   
           JPX9997     0.000000  0.000000  0.000000  0.000000  0.000000  ...   

                                                                         \
                        VOLUME8   VOLUME7   VOLUME6   VOLUME5   VOLUME4   
datetime   instrument                                                     
2021-12-01 JPX1301     0.550156 -0.515034 -0.818786  0.219389 -0.712340   
           JPX1332    -0.041365 -0.304918 -0.551348 -0.769181 -0.923758   
           JPX1333    -0.232837 -0.670710 -0.622410 -0.313527 -0.727769   
           JPX1375    -0.882427 -0.778017 -0.853102 -0.415059 -0.649966   
           JPX1376    -1.346688 -0.908822 -0.538273  0.005606  0.046076   
...                         ...       ...       ...       ...       ...   
2021-12-06 JPX9990     0.000000  0.000000  0.000000  0.000000  0.000000   
           JPX9991     0.000000  0.000000  0.000000  0.000000  0.000000   
           JPX9993     0.000000  0.000000  0.000000  0.000000  0.000000   
           JPX9994     0.000000  0.000000  0.000000  0.000000  0.000000   
           JPX9997     0.000000  0.000000  0.000000  0.000000  0.000000   

                                                             \
                        VOLUME3   VOLUME2   VOLUME1 VOLUME0   
datetime   instrument                                         
2021-12-01 JPX1301     0.131457  0.613940  1.943128     0.0   
           JPX1332    -0.358178 -0.096569  1.328281     0.0   
           JPX1333    -0.318858  0.282089  0.845876     0.0   
           JPX1375    -0.473393 -0.902544  0.138783     0.0   
           JPX1376     0.091182  2.091851  3.000000     0.0   
...                         ...       ...       ...     ...   
2021-12-06 JPX9990     0.000000  0.000000  0.000000     0.0   
           JPX9991     0.000000  0.000000  0.000000     0.0   
           JPX9993     0.000000  0.000000  0.000000     0.0   
           JPX9994     0.000000  0.000000  0.

In [ ]:
print(prices.loc[prices.loc[:, 'instrument']=='jpx1301'])
# prices

        date          RowId instrument    open    high     low   close  \
0 2021-12-06  20211206_1301    jpx1301  2982.0  2982.0  2965.0  2971.0   

   volume  factor  ExpectedDividend  SupervisionFlag  
0    8900     1.0               NaN            False  


In [ ]:
# update predictions
updater = PredUpdater(model, recorder, to_date=end_date)
updated_data = updater.update(dataset=dataset, ret_new=True, write=False)
print(updated_data)

[6202:MainThread](2022-06-03 22:55:00,542) WARNING - qlib.PredUpdater - [update.py:155] - The given `to_date`(2021-12-06 00:00:00) is later than `latest_date`(2021-12-06 00:00:00). So `to_date` is clipped to `latest_date`.
[6202:MainThread](2022-06-03 22:55:00,772) INFO - qlib.PredUpdater - [1218860721.py:43] - Finish updating new 2000 predictions in 491c1471eaba426e9a9aa067b543df9c.


                          score
datetime   instrument          
2018-08-01 JPX1301     0.010234
           JPX1332     0.011423
           JPX1333     0.002717
           JPX1376     0.011578
           JPX1377     0.006518
...                         ...
2021-12-06 JPX9990     0.012965
           JPX9991     0.012965
           JPX9993     0.012965
           JPX9994     0.012965
           JPX9997     0.012965

[240144 rows x 1 columns]
